# Ground truth dataset
This code access our GitHub, where benchmark is storaged and create a DataFrame (url, label)

In [2]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

In [6]:
repo_owner = 'KvindtEva'
repo_name = 'Time-Machine-Marburg'
access_token = 'your_access_token'

In [7]:
base_api_url = f'https://api.github.com/repos/{repo_owner}/{repo_name}/contents/'

# Function to get folder contents with authentication
def get_folder_contents(folder_url):
    response = requests.get(folder_url, headers={'Authorization': f'token {access_token}'})
    response.raise_for_status()  # Check for request errors
    return response.json()
def get_image_links_and_folders(base_url):
    # Get the contents of the root directory
    root_contents = get_folder_contents(base_url)
    
    data = []
    
    # Iterate through each item in the root directory
    for item in root_contents:
        if item['type'] == 'dir':  # Check if the item is a directory
            folder_url = item['url']
            folder_name = item['name']
            
            # Get contents of the folder
            folder_contents = get_folder_contents(folder_url)
            
            for file_item in folder_contents:
                if file_item['type'] == 'file' and file_item['name'].lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
                    image_url = file_item['download_url']
                    data.append([image_url, folder_name])
    
    return data

In [8]:
image_folder_path = 'datasets_benchmark'
folder_url = base_api_url + image_folder_path

folder_url = folder_url[:-18] + "DATASET_GROUNDTRUTH"

# Get data and create DataFrame
data = get_image_links_and_folders(folder_url)
df = pd.DataFrame(data, columns=['url', 'label'])

df.head()

,url,label
0,https://raw.githubusercontent.com/KvindtEva/Ti...,Elisabethkirche_DeutscherOrden
1,https://raw.githubusercontent.com/KvindtEva/Ti...,Elisabethkirche_DeutscherOrden
2,https://raw.githubusercontent.com/KvindtEva/Ti...,Elisabethkirche_DeutscherOrden
3,https://raw.githubusercontent.com/KvindtEva/Ti...,Elisabethkirche_DeutscherOrden
4,https://raw.githubusercontent.com/KvindtEva/Ti...,Elisabethkirche_DeutscherOrden


In [9]:
df.to_csv('datasets/production/dataset_benchmark.csv', index=False)